### Jul AdventKalender D16

https://adventofcode.com/2022/day/16

In [1]:
import numpy as np
import re

#### Day 16.1 

Given a scan output of a network of pipes and pressure-release valves. It contains each valve's flow rate if it were opened (in pressure per minute) and the tunnels you could use to move between the valves. 

    Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
    Valve BB has flow rate=13; tunnels lead to valves CC, AA
    Valve CC has flow rate=2; tunnels lead to valves DD, BB
    ......

Start with Valve AA, it will take **one minute to open** a single valve and **one minute to travel** from one valve to another. With a total of **30 minutes**, what is the most pressure you could release?

In [2]:
def readOutput(file_name):
    data_valves = {}
    f = open(file_name, "r")
    while True:
        line = f.readline()
        if not line:
            break
        valve = re.findall(r"^Valve (\w+).*rate=(\d+).*(?:valves|valve) (.*)", line.strip())[0]
        data_valves[valve[0]] = {'rate':int(valve[1]), 'tunnels':valve[2].split(', ')}
    f.close()
    return data_valves

In [3]:
# build paths from every valve to ather and save the cost(steps/minutes)
def buildPaths(data_valves):
    paths = {}
    for val_start in data_valves.keys():
        queue = [val_start]
        distances = {val_start:0}
        visited = set(val_start)
        while len(queue)>0:
            val_cur = queue.pop(0)
            dis = distances[val_cur]
            for tunnel in data_valves[val_cur]['tunnels']:
                if tunnel not in visited:
                    visited.add(tunnel)
                    distances[tunnel] = dis + 1
                    queue.append(tunnel)
        paths[val_start] = distances
    return paths

In [4]:
# filter the valves and keep only valves with positive rates
def filterPosRates(data_valves):
    return [key for key,valve in data_valves.items() if valve['rate'] > 0]

In [5]:
# each valve with positive rate can have 2 states: open or close
# there will then be in total 2^len(valves_pos) states (will be constructed in bit/binary representation)
# for every minute and every current valve(positive rate), it corresponds to one combination of states
# represented by a matrix(list) of 30 x len(valves_pos) (index of valves_pos) x 2^len(valves_pos): pressure
def _genStates(len_pos, minutes, init_value):
    res = []
    for _ in range(minutes):
        loc_states = []
        for _ in range(len_pos):
            loc_states.append([init_value] * (1<<len_pos))
        res.append(loc_states)
    return res

def initialStates(valves_pos, minutes, valve_paths):
    init_states = _genStates(len(valves_pos), minutes, init_value=float(-np.inf))
    val_start = valve_paths
    # distances between valves can be mapped to minutes axis, because 1 step = 1 minute
    for i in range(len(valves_pos)):
        # distance/minute cost between start to every positive valve
        dist = valve_paths['AA'][valves_pos[i]]
        minutes_cost = dist + 1 # minutes cost/steps/distance from start AA
        init_states[minutes_cost][i][1<<i]=0 # set 0 to states when ONE positive valve is ON
    return init_states

In [12]:
# get pressure of input state, by index in the state list - (0,2^len(valves_pos))
def _getPressure(state_id, valves_pos, data_valves):
    res=0
    for i in range(len(valves_pos)):
        if 1<<i & state_id != 0:
            res+= data_valves[valves_pos[i]]['rate']
    return res

def buildStatesTable(min_total, states_table, valves_pos, data_valves, valve_paths):
    for m in range(1,min_total+1): # each minute
        for i in range(len(valves_pos)): #  each current valve
            for j in range(1<<len(valves_pos)): # each possible state
                # if last or current minute is not -inf
                if states_table[m-1][i][j] == float(-np.inf) and states_table[m][i][j] == float(-np.inf):
                    continue
                pres = _getPressure(j, valves_pos, data_valves) # get pressure for each state and set to a proper (min,place)
                if states_table[m-1][i][j] != float(-np.inf): # if last minute is not -inf
                    state_cur = states_table[m-1][i][j] + pres
                    if state_cur > states_table[m][i][j]:
                        states_table[m][i][j] = state_cur
                if 1<<i & j == 0: # current valve i is not ON in the current state j
                    continue
                 # current valve i is ON in the current state j
                if states_table[m][i][j] == float(-np.inf):
                    continue
                for p in range(len(valves_pos)): # states ps when EACH ONE of the positive valves is ON 
                    if 1<<p & j !=0: # state p is already included in the current state
                        continue
                    dist = valve_paths[valves_pos[i]][valves_pos[p]] # dist to all positive valves from valves_pos[i]
                    minutes_cost = dist + 1 
                    if m + minutes_cost >= min_total+1: # out of range
                        continue
                    state_cur = states_table[m][i][j] + pres*minutes_cost #generate every minute
                    if state_cur > states_table[m+minutes_cost][p][j|(1<<p)]:
                        states_table[m+minutes_cost][p][j|(1<<p)] = state_cur # set to next minute

In [13]:
min_total = 30
# read data
data_valves = readOutput("data/input16.txt")
# build path using A*, only save destinations and distances
valve_paths = buildPaths(data_valves)
# all valves with positive rate
valves_pos = filterPosRates(data_valves)
# initialize state table
states_table = initialStates(valves_pos, min_total+1, valve_paths)
# construct state table
buildStatesTable(min_total, states_table, valves_pos, data_valves, valve_paths)
# state table                  
max([max(valve) for valve in states_table[30]])

2320

#### Day 16.2

With you and an elephant who you taught using 4 minutes working together for **26** minutes, what is the most pressure you could release?

In [18]:
pressure_two = 0
# can directly calculate from the resulting states_table
for i in range(1<<len(valves_pos)): # each state
    for j in range(1<<len(valves_pos)): # each state
        # if ON valves in state j are not all ON in state i 
        # (when cooperating, one's state has to be a subset of another)
        if i & j != j: 
            continue
        person1 = float(-np.inf)
        person2 = float(-np.inf)
        # check each positive valve, choose the max combination
        for p in range(len(valves_pos)):
            person1 = max(person1, states_table[26][p][j]) # state j
            person2 = max(person2, states_table[26][p][i&~j]) # state i but exclude state j
        # save the larger result
        pressure_two = max(pressure_two, person1+person2)
pressure_two

2967